# Assignment 4
* Unit: SIT720
* Name: Bryon Baker
* Student #: 85031775

In [1]:
#!pip3 install pandas
#!pip3 install numpy

In [13]:
import sys
import warnings
import random
import pandas as pd # dataframe manipulation
import numpy as np # linear algebra

from sklearn.metrics import mean_squared_error

In [14]:
# Seed the random nunmber generator to ensure K-Means randomisation works properly.
random.seed()

print(f"Seeding random number generator. Random number is: {random.random()}")

Seeding random number generator. Random number is: 0.5824216833999516


In [15]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass4/forestfires.csv'
full_df = pd.read_csv(url)
print(f"{full_df.head()}\n")

   X  Y month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain  area
0  7  5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   0.0
1  7  4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   0.0
2  7  4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   0.0
3  8  6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   0.0
4  8  6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   0.0



# Pre Process the Data
1. One hot encode the day and month attributes.
2. ...

In [16]:
# Check the dataset has no NaNs.
# TODO: Check the dtaa quality

if full_df.isnull().sum().sum() == 0:
    print("Data does not have any missing values.")
else:
    print("Data has missing values and needs pre processing.")
    print("Stopping....")
    quit() 

Data does not have any missing values.


In [17]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', drop=None, dtype='int')

In [18]:
# One-hot encode month
transformed = ohe.fit_transform(full_df[['month']])
month_df = pd.DataFrame(transformed.toarray())
month_df.columns = ohe.categories_[0]

In [19]:
# One hot encode day
transformed = ohe.fit_transform(full_df[['day']])
day_df = pd.DataFrame(transformed.toarray())
day_df.columns = ohe.categories_[0]

In [20]:
encoded_df = full_df.drop(['month','day'], axis=1)

del full_df         # Make sure we don't accidentally use full_df later

In [21]:
encoded_df = pd.concat([encoded_df, month_df], axis=1)
encoded_df = pd.concat([encoded_df, day_df], axis=1)
encoded_df.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,nov,oct,sep,fri,mon,sat,sun,thu,tue,wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,1,0,0,0,0,0,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,1,0,0,0,0,0,0,1,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,1,0,0,0,1,0,0,0,0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,1,0,0,0,0,0,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,0,0,1,0,0,0


## Preprocess area
Reduce the skewness of area as per procedure on page 5.
This will be used to train the model, but it is important to take the inverse log after prediction so as to find the predicted area. 


In [37]:
encoded_df["xfrmd_area"] = encoded_df["area"] +1                 # Create a new area column that will contain the ln(area+1)
print(encoded_df[["area","xfrmd_area"]])
encoded_df["xfrmd_area"] = np.log(encoded_df['xfrmd_area'])      # Take the ln(area+1)
print(encoded_df[["xfrmd_area"]])                                # Have a look and make sure it worked....

      area  xfrmd_area
0     0.00        1.00
1     0.00        1.00
2     0.00        1.00
3     0.00        1.00
4     0.00        1.00
..     ...         ...
512   6.44        7.44
513  54.29       55.29
514  11.16       12.16
515   0.00        1.00
516   0.00        1.00

[517 rows x 2 columns]
     xfrmd_area
0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
..          ...
512    2.006871
513    4.012592
514    2.498152
515    0.000000
516    0.000000

[517 rows x 1 columns]


## Create Sub-Datasets

Create the sub datasets that the different models will need.

# Create the Results DataFrame

In [38]:
results_df = pd.DataFrame(columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])

# Calculate the Naive results

In [42]:
# Generate an array with the predicted values being the mean of the area
y = encoded_df["area"]
y_pred = np
y_naive_pred = np.repeat(y, len(y))

# Calculate the naive RMSE
naive_rmse = mean_squared_error(y, y_naive_pred, squared=False)
print(naive_rmse)

# Add the naive results to the Results Table
rmse_df = pd.DataFrame([['Naive', naive_rmse, naive_rmse, naive_rmse, naive_rmse]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df.append(rmse_df)

ValueError: Found input variables with inconsistent numbers of samples: [517, 267289]